# Statistics on the output data. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

df = pd.read_csv('data/output-tunslip.csv')

if not os.path.exists("figure"):
    os.mkdir("figure")

df.columns

Index(['File', 'Line', 'Column', 'DefaultLevel', 'Level', 'Category', 'Name',
       'Warning', 'Suggestion', 'Note', 'CWEs', 'Context', 'Fingerprint',
       'ToolVersion', 'RuleId', 'HelpUri'],
      dtype='object')

In [9]:
dfp = df[['Category', 'Name', 'CWEs']].groupby(['Category', 'Name', 'CWEs'], group_keys=False).size().reset_index(name='Count')
print(dfp)
fig = px.sunburst(dfp, path=['Category', 'Name', 'CWEs'], values='Count')
# fig.write_image("figure/vul_statistics.pdf")
fig.show()
df.head(3)

    Category       Name              CWEs  Count
0     buffer       char  CWE-119!/CWE-120      6
1     buffer     getopt   CWE-120, CWE-20      1
2     buffer     memcpy           CWE-120     11
3     buffer       read   CWE-120, CWE-20      1
4     buffer     strcat           CWE-120      1
5     buffer     strcpy           CWE-120      4
6     buffer    strncpy           CWE-120      1
7     format  vsnprintf           CWE-134      1
8    integer       atoi           CWE-190      2
9       misc       open           CWE-362      2
10  obsolete     usleep           CWE-676      1
11     shell     system            CWE-78      1


,File,Line,Column,DefaultLevel,Level,Category,Name,Warning,Suggestion,Note,CWEs,Context,Fingerprint,ToolVersion,RuleId,HelpUri
0,data/projects/contiki-2.4/tools/tunslip.c,438,3,4,4,format,vsnprintf,If format strings can be influenced by an atta...,Use a constant for the format specification.,NaN,CWE-134,"vsnprintf(cmd, sizeof(cmd), fmt, ap);",5a6ebc53325719b0f59e24f31e308667e9337289bfbc55...,2.0.19,FF1019,https://cwe.mitre.org/data/definitions/134.html
1,data/projects/contiki-2.4/tools/tunslip.c,442,10,4,4,shell,system,This causes a new program to execute and is di...,try using a library call that implements the s...,NaN,CWE-78,return system(cmd);,f3f1fc4af80877aebfc643d7ec0da9b9e96eccff3cad56...,2.0.19,FF1044,https://cwe.mitre.org/data/definitions/78.html
2,data/projects/contiki-2.4/tools/tunslip.c,753,3,4,4,buffer,strcat,Does not check for buffer overflows when conca...,"Consider using strcat_s, strncat, strlcat, or ...",NaN,CWE-120,"strcat(t, dev);",901ebb10ee39a0103c1035a30e7a1a1f2db2bf8d9bf07e...,2.0.19,FF1005,https://cwe.mitre.org/data/definitions/120.html


## Grepping functions from the vulnerability context of the file.

In [14]:
file = df.File[0]
xml_file = "a.cpp.xml"

file_str = ''
with open(file) as f:
    file_str = f.read()

In [15]:
import subprocess as sub
from pylibsrcml import srcml

# file_xml = sub.run(['srcml', file])
srcml.srcml(file, "a.cpp.xml")

In [86]:
fun_ptn = "string(//src:function)"
funblk_ptn = "//src:function/src:name"
file_ptn = "string(//src:unit/@filename)" 

cmd = sub.Popen(["srcml", "--xpath", fun_ptn, file], stderr=sub.STDOUT)
out, err = cmd.communicate()

void
relay_dhcp_to_server(struct ip *ip, int len)
{
  struct dhcp_light_msg *inm;
  struct dhcp_msg m;
  int n;
  u_int8_t *optptr;
  
  inm = (void*)(((u_int8_t*)ip) + 20 + 8); /* Skip over IP&UDP headers. */

  if (inm->op != BOOTREQUEST) {
    return;
  }

  inm->flags = ntohs(BOOTP_BROADCAST);

  memcpy(&m, inm, DHCP_BASE_LEN);
  memset(&m.sname, 0x0, DHCP_HOLE_LEN);
  memcpy(&m.options, &inm->options, len - 20 - 8 - DHCP_BASE_LEN);
  n = (len - 20 - 8) + DHCP_HOLE_LEN; /* +HOLE -IP&UDP headers. */

  /*
   * Ideally we would like to use the Relay Agent information option
   * (RFC3046) together with the Link Selection sub-option (RFC3527)
   * to ensure that addresses are allocated for this
   * subnet. Unfortunately ISC-DHCPD does not currently implement
   * RFC3527 and some other mechanism must be used. For this reason
   * this implementation in addition uses the DHCP option for subnet
   * selection (RFC3011) which is really not intended to be used by
   * relays.
   *
   * F

In [51]:
cmd = sub.Popen(["srcml", "--xpath", funblk_ptn, xml_file], stderr=sub.STDOUT)
out, err = cmd.communicate()

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<unit xmlns="http://www.srcML.org/srcML/src" revision="1.0.0">

<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="1.0.0" language="C" filename="data/projects/contiki-2.4/tools/tunslip.c" item="1"><name>relay_dhcp_to_server</name></unit>

<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="1.0.0" language="C" filename="data/projects/contiki-2.4/tools/tunslip.c" item="2"><name>relay_dhcp_to_client</name></unit>

<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="1.0.0" language="C" filename="data/projects/contiki-2.4/tools/tunslip.c" item="3"><name>ip4sum</name></unit>

<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="1.0.0" language="C" filename="data/projects/contiki-2.4/tools/tunslip.c" item="4"><name>check_ip</name></unit>

<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="1.0.0" language="C" filename="data/projects/contiki-2.4/tools/tunslip.c" item="5"><name>is_sensible_string</name></